# Setup

In [1]:
# TODO: add annotations describing usage of different modules

from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import altair as alt
import random
import warnings

warnings.filterwarnings('ignore')

# set seaborn whitegrid theme
sns.set(style="whitegrid")

from sklearn.inspection import permutation_importance
from random import sample


/Users/orencarmeli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')
print(cdc_survey.shape)

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
print(cdc_survey_pmom.shape)

(35706, 863)
(7741, 863)


In [3]:
# Depression screener
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]

# Alcohol & smoking
alcohol_n_smoking_cols = [
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo', 
    'drank_alc',
    'alc_drinking_freq',
    'alc_per_day',
    'times_with_4or5_alc',
    'times_with_8plus_alc',
    'times_with_12plus_alc',
    '4plus_alc_daily',
    'days_4plus_drinks_occasion',
    'smoked_100_cigs',
    'currently_smoke'
]

#Medical Conditions 
med_condition_cols = [
    'chest_discomfort',
    'asthma',
    'anemia_treatment',
    'blood_transfusion',
    'arthritis',
    'heart_failure',
    'coronary_heart_disease',
    'angina_pectoris',
    'heart_attack',
    'stroke',
    'thyroid_issues',
    'respiratory_issues',
    'abdominal_pain',
    'gallstones',
    'gallbladder_surgery',
    'cancer',
    'dr_recommend_lose_weight',
    'dr_recommend_exercise',
    'dr_recommend_reduce_salt',
    'dr_recommend_reduce_fat',
    'currently_losing_weight',
    'currently_increase_exercise',
    'currently_reducing_salt',
    'currently_reducing_fat',
    'metal_objects',
    'has_diabetes',    
    #Blood Pressure & Cholesterol 
    'high_bp',
    'age_hypertension',
    'hypertension_prescription',
    'high_bp_prescription',
    'high_cholesterol',
    'cholesterol_prescription',
    # general health
    'has_overweight_diagnosis',   
    'height_in',
    'weight_lbs', 
    'general_health_condition',      
]

# diet, nutrition, & exercise 
lifestyle_cols = [
    # exercise/weight loss
    'vigorous_work',
    'walk_or_bicycle',
    'vigorous_recreation',
    'moderate_recreation',    
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo', 
    'attempt_weight_loss_1yr',           
    # diet
    'how_healthy_is_your_diet',    
    'milk_consumption_freq',
    'govmnt_meal_delivery',
    'nonhomemade_meals',
    'fastfood_meals',
    'readytoeat_meals',
    'frozen_pizza'   
]

# Reproductive Health 
reproductive_health_cols = [
    'regular_periods',
    'age_last_period',
    'try_pregnancy_1yr',
    'see_dr_fertility',
    'pelvic_infection',
    'pregnant_now',
    'pregnancy_count',
    'diabetes_pregnancy',
    'delivery_count',
    'live_birth_count',
    'age_at_first_birth',
    'age_at_last_birth',
    'months_since_birth',
    'horomones_not_bc'
]

# hospital usage & access to care
hospital_utilization_n_access_cols = [
    'general_health',
    'regular_healthcare_place',
    'time_since_last_healthcare',
    'overnight_in_hospital',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',   
    'have_health_insurance',
    'have_private_insurance',
    'has_health_insurance_gap',       
    'plan_cover_prescriptions'    
]

# socioeconomic status
socioeconomic_cols = [
    #Food Security
    'emergency_food_received',
    'food_stamps_used',      
    #Income
    'family_poverty_level',
    'family_poverty_level_category',
    #Occupation
    'hours_worked',
    'over_35_hrs_worked',
    'work_schedule'    
]

demographic_cols = [
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born'
]

# combine all candidate features
candidate_features = dep_screener_cols + \
    alcohol_n_smoking_cols + \
    med_condition_cols + \
    hospital_utilization_n_access_cols + \
    lifestyle_cols + \
    socioeconomic_cols + \
    reproductive_health_cols + \
    demographic_cols


len(candidate_features)

115

In [4]:
## ###############################################################################
## function to create test & train dataset

def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False,
                   return_indices=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    #cols_to_use.insert(0, 'SEQN')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    indices = np.arange(y.shape[0])
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split(
        x, 
        y, 
        indices,
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    if return_indices:
        return x_train, x_test, y_train, y_test, idx_train, idx_test
    else:
        return x_train, x_test, y_train, y_test

## ###############################################################################
## Function to get performance across different models

def get_performance_df(label_actual, label_pred, model_name):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    # create classification report
    result_table = classification_report(label_actual, label_pred, output_dict=True)
    result_table = pd.DataFrame.from_dict(result_table)

    # store for later
    accuracies = result_table['accuracy'][0]
    
    column_key = {
        '0':'Depressed (No)',
         '1':'Depressed (Yes)',
         'accuracy':'accuracy',
         'macro avg':'Macro Avg',
         'weighted avg':'Weighted Avg'
    }

    # rename grouping
    result_table.columns = [column_key.get(key) for key in result_table.columns]

    # create dataframe with 1 row per grouping
    result_table.drop(labels = 'accuracy', axis = 1, inplace=True)
    result_table = result_table.transpose()
    result_table['accuracy'] = [accuracies for i in range(result_table.shape[0])]
    result_table = result_table.reset_index()
    result_table.rename(columns = {'index':'grouping'},inplace=True)
    result_table['model'] = model_name
    result_table = result_table[['model','grouping','precision','recall','f1-score','support','accuracy']]
    return result_table

In [5]:
cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)

# Opt 1: Best Single Model (Include All Observations)


In [7]:
model_features_opt1 = dep_screener_cols + ['times_with_12plus_alc',
 'seen_mental_health_professional',
 'count_days_seen_doctor_12mo',
 'count_lost_10plus_pounds',
 'arthritis',
 'horomones_not_bc',
 'is_usa_born',
 'times_with_8plus_alc',
 'time_since_last_healthcare',
 'duration_last_healthcare_visit',
 'work_schedule',
 'age_in_years',
 'regular_periods',
 'count_minutes_moderate_sedentary_activity',
 'emergency_food_received',
 'high_bp',
 'dr_recommend_exercise',
 'metal_objects',
 'drank_alc',
 'cholesterol_prescription',
 'smoked_100_cigs',
 'vigorous_recreation',
 'dr_recommend_lose_weight',
 'cancer',
 'chest_discomfort',
 'have_health_insurance',
 'weight_lbs',
 'high_cholesterol']

x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = model_features_opt1,
    return_indices = True
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

gnb_opt1 = GaussianNB()
gnb_opt1.fit(x_train, y_train)
gnb_opt1_pred  = gnb_opt1.predict(x_test)
gnb_opt1_score = get_performance_df(y_test, gnb_opt1_pred,'Gaussian Naive Bayes')  
gnb_opt1_score

(6192, 38)
(1549, 38)
(6192,)
(1549,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes,Depressed (No),0.951777,0.809353,0.874806,1390.0,0.792124
1,Gaussian Naive Bayes,Depressed (Yes),0.277929,0.641509,0.387833,159.0,0.792124
2,Gaussian Naive Bayes,Macro Avg,0.614853,0.725431,0.631319,1549.0,0.792124
3,Gaussian Naive Bayes,Weighted Avg,0.882608,0.792124,0.824819,1549.0,0.792124


**Performance on those with 1+ dep screener answered**

In [8]:
gnb_opt1_pred_prob = pd.DataFrame(gnb_opt1.predict_proba(x_test))
gnb_opt1_pred_prob.columns = ['depressed_no_prob','depressed_yes_prob']
gnb_opt1_pred_prob['y_pred'] = gnb_opt1_pred
gnb_opt1_pred_prob['y_actual'] = y_test

gnb_opt1_pred_prob_with_feat = pd.concat([
    gnb_opt1_pred_prob, 
    # column bind test observations with their original values
    cdc_survey_pmom.iloc[idx_test][model_features_opt1].reset_index()
], axis=1)

gnb_opt1_pred_prob_with_feat['num_dep_screener_0'] = (gnb_opt1_pred_prob_with_feat[dep_screener_cols]==0).sum(axis=1)

In [9]:
replied_to_1plus_dep_screener = gnb_opt1_pred_prob_with_feat[
    gnb_opt1_pred_prob_with_feat['num_dep_screener_0'] < 9
]

get_performance_df(
    replied_to_1plus_dep_screener['y_actual'], 
    replied_to_1plus_dep_screener['y_pred'],
    'GNB: Only those with 1 dep screener replied'
)  

,model,grouping,precision,recall,f1-score,support,accuracy
0,GNB: Only those with 1 dep screener replied,Depressed (No),0.929104,0.675712,0.782404,737.0,0.680139
1,GNB: Only those with 1 dep screener replied,Depressed (Yes),0.275758,0.705426,0.396514,129.0,0.680139
2,GNB: Only those with 1 dep screener replied,Macro Avg,0.602431,0.690569,0.589459,866.0,0.680139
3,GNB: Only those with 1 dep screener replied,Weighted Avg,0.831781,0.680139,0.724921,866.0,0.680139


# Opt 2: Best Single Model (Only Observations with 1+ dep screener resposnse)

In [6]:
cdc_survey_pmom['sum_dep_screener_top2'] = \
   cdc_survey_pmom['difficult_doing_daytoday_tasks'] + cdc_survey_pmom['feeling_down_depressed_hopeless']

model_features_opt2 = dep_screener_cols + [
    'seen_mental_health_professional',
    'times_with_12plus_alc',
    'duration_last_healthcare_visit',
    'time_since_last_healthcare',
    'cholesterol_prescription',
    'high_cholesterol',
    'age_in_years',
    'horomones_not_bc',
    'times_with_8plus_alc',
    'months_since_birth',
    'arthritis',
    'high_bp',
    'regular_periods',
    'moderate_recreation',
    'thyroid_issues',
    'vigorous_recreation',
    'stroke',
    'is_usa_born',
    'sum_dep_screener_top2'
    # 'asthma'
]

x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9],
    columns = model_features_opt2,
    return_indices = True
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

gnb_opt2 = GaussianNB()
gnb_opt2.fit(x_train, y_train)
gnb_opt2_pred  = gnb_opt2.predict(x_test)
gnb_opt2_score = get_performance_df(y_test, gnb_opt2_pred,'Gaussian Naive Bayes (Subset)')  
gnb_opt2_score

(3515, 29)
(879, 29)
(3515,)
(879,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes (Subset),Depressed (No),0.940754,0.709066,0.808642,739.0,0.717861
1,Gaussian Naive Bayes (Subset),Depressed (Yes),0.332298,0.764286,0.463203,140.0,0.717861
2,Gaussian Naive Bayes (Subset),Macro Avg,0.636526,0.736676,0.635923,879.0,0.717861
3,Gaussian Naive Bayes (Subset),Weighted Avg,0.843844,0.717861,0.753623,879.0,0.717861


## Callout

There are 5 features in op2 not in opt1. If we use these as ensemble will need to think about how we deal with this. I.E maybe we can market it where we start with 10 dep screener cols. Based on their response we use either X or Y additional cols.

In [11]:
[feature for feature in model_features_opt2 if feature not in model_features_opt1]

['months_since_birth',
 'moderate_recreation',
 'thyroid_issues',
 'stroke',
 'sum_dep_screener_top2']

# Opt 3: Ensemble Model V1

Build 2 models both use feature set from model opt 1
- Model 1: trained on the entire dataset
- Model 2: trained on observations where the respondent answers 0 for 9+ of the depression screeners

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [24]:
def get_model_data_ensemble_v1(original_df, 
                            columns_to_use):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    data_low_dep_screener = original_df[original_df['num_dep_screener_0'] >= 9].copy()
    data_high_dep_screener = original_df[original_df['num_dep_screener_0'] < 9].copy()
    
    x_train_low, x_test_low, y_train_low, y_test_low = get_model_data(
        original_df = data_low_dep_screener,
        columns = columns_to_use
    )
    
    x_train_high, x_test_high, y_train_high, y_test_high = get_model_data(
        original_df = data_high_dep_screener,
        columns = columns_to_use
    )
    
    return x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high

x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v1(
    original_df = cdc_survey_pmom,
    columns_to_use = model_features_opt1
)

print(x_train_low.shape)
print(x_test_low.shape)
print(y_train_low.shape)
print(y_test_low.shape)
print(x_train_high.shape)
print(x_test_high.shape)
print(y_train_high.shape)
print(y_test_high.shape)

################# Model 1: Based on all datapoints
x_train_all = np.concatenate((x_train_low, x_train_high))
y_train_all = np.concatenate((y_train_low, y_train_high))

gnb = GaussianNB()
gnb.fit(x_train_all, y_train_all)

################# Model 2: Based on cases where >8 dep screener are 0
dt = DecisionTreeClassifier(random_state=42)
dt.fit(x_train_low, y_train_low)

################# Predict 

# Model 1: use for high cases
y_pred_high = gnb.predict(x_test_high)

# Model 2: use for low cases
y_pred_low = dt.predict(x_test_low)

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

ensemble_performance_opt3 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt3[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(2677, 38)
(670, 38)
(2677,)
(670,)
(3515, 38)
(879, 38)
(3515,)
(879,)


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.947368,0.822335,0.880435,0.801162,1379.0
1,All Obs,Depressed (Yes),0.303977,0.629412,0.409962,0.801162,170.0
2,All Obs,Macro Avg,0.625673,0.725873,0.645198,0.801162,1549.0
3,All Obs,Weighted Avg,0.876757,0.801162,0.828801,0.801162,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.961240,0.968750,0.964981,0.932836,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.200000,0.166667,0.181818,0.932836,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.580620,0.567708,0.573399,0.932836,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.927155,0.932836,0.929914,0.932836,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.931159,0.695535,0.796282,0.700796,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.311927,0.728571,0.436831,0.700796,140.0


# Opt 4: Ensemble Model V2

Build 2 models with different feature set
- Model 1: trained on observations with 1+ dep screener response. Uses features from opt 2
- Model 2: trained on observations with 0 dep screener response. Uses features from opt 1

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [23]:
def get_model_data_ensemble_v2(original_df, 
                            columns_to_use_low,
                            columns_to_use_high):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    data_low_dep_screener = original_df[original_df['num_dep_screener_0'] >= 9].copy()
    data_high_dep_screener = original_df[original_df['num_dep_screener_0'] < 9].copy()
    
    x_train_low, x_test_low, y_train_low, y_test_low = get_model_data(
        original_df = data_low_dep_screener,
        columns = columns_to_use_low
    )
    
    x_train_high, x_test_high, y_train_high, y_test_high = get_model_data(
        original_df = data_high_dep_screener,
        columns = columns_to_use_high
    )
    
    return x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high

x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_opt1,
    columns_to_use_high = model_features_opt2
)

print(x_train_low.shape)
print(x_test_low.shape)
print(y_train_low.shape)
print(y_test_low.shape)
print(x_train_high.shape)
print(x_test_high.shape)
print(y_train_high.shape)
print(y_test_high.shape)
print(len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high))

## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

gnb = GaussianNB()
gnb.fit(x_train_high, y_train_high)
y_pred_high = gnb.predict(x_test_high)

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

dt = DecisionTreeClassifier(random_state=42)
dt.fit(x_train_low, y_train_low)
y_pred_low = dt.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt4 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt4[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(2677, 38)
(670, 38)
(2677,)
(670,)
(3515, 29)
(879, 29)
(3515,)
(879,)
7741


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.951747,0.829587,0.886478,0.810846,1379.0
1,All Obs,Depressed (Yes),0.322767,0.658824,0.433269,0.810846,170.0
2,All Obs,Macro Avg,0.637257,0.744205,0.659873,0.810846,1549.0
3,All Obs,Weighted Avg,0.882718,0.810846,0.836739,0.810846,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.961240,0.968750,0.964981,0.932836,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.200000,0.166667,0.181818,0.932836,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.580620,0.567708,0.573399,0.932836,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.927155,0.932836,0.929914,0.932836,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.940754,0.709066,0.808642,0.717861,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.332298,0.764286,0.463203,0.717861,140.0


# Opt 5: Ensemble Model V3

Build 2 models with different feature set
- Model 1: trained on observations with 1+ dep screener response. Uses features from opt 2
- Model 2: trained on observations with 0 dep screener response. Uses separate features

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [16]:
model_features_low = [
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'work_schedule',
    'age_in_years'
]

def get_model_data_ensemble_v2(
    original_df, 
    columns_to_use_low,
    columns_to_use_high):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    data_low_dep_screener = original_df[original_df['num_dep_screener_0'] >= 9].copy()
    data_high_dep_screener = original_df[original_df['num_dep_screener_0'] < 9].copy()
    
    x_train_low, x_test_low, y_train_low, y_test_low = get_model_data(
        original_df = data_low_dep_screener,
        columns = columns_to_use_low
    )
    
    x_train_high, x_test_high, y_train_high, y_test_high = get_model_data(
        original_df = data_high_dep_screener,
        columns = columns_to_use_high
    )
    
    return x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high

x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_low,
    columns_to_use_high = model_features_opt2
)

print(x_train_low.shape)
print(x_test_low.shape)
print(y_train_low.shape)
print(y_test_low.shape)
print(x_train_high.shape)
print(x_test_high.shape)
print(y_train_high.shape)
print(y_test_high.shape)
print(len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high))

## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

gnb = GaussianNB()
gnb.fit(x_train_high, y_train_high)
y_pred_high = gnb.predict(x_test_high)

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)

gb = GradientBoostingClassifier(random_state=42)
gb.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = gb.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt5 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt5[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(2677, 12)
(670, 12)
(2677,)
(670,)
(3515, 29)
(879, 29)
(3515,)
(879,)
7741


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.954925,0.829587,0.887854,0.813428,1379.0
1,All Obs,Depressed (Yes),0.330484,0.682353,0.445298,0.813428,170.0
2,All Obs,Macro Avg,0.642705,0.755970,0.666576,0.813428,1549.0
3,All Obs,Weighted Avg,0.886394,0.813428,0.839284,0.813428,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.967239,0.968750,0.967994,0.938806,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.310345,0.300000,0.305085,0.938806,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.638792,0.634375,0.636539,0.938806,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.937826,0.938806,0.938311,0.938806,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.940754,0.709066,0.808642,0.717861,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.332298,0.764286,0.463203,0.717861,140.0


# Opt 6: Ensemble Model V4

Build 2 models with different feature set
- Model 1: trained on observations with 1+ dep screener response. Uses features from opt 2
- Model 2: trained on observations with 0 dep screener response. Uses separate features

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [17]:
model_features_low = [
    'respiratory_issues',
    'seen_mental_health_professional',
    'walk_or_bicycle',
    'arthritis',
    'has_smoked_tabacco_last_5days',
    'work_schedule',
    'currently_increase_exercise',
    'over_35_hrs_worked',
    'chest_discomfort'
]

def get_model_data_ensemble_v2(
    original_df, 
    columns_to_use_low,
    columns_to_use_high):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    data_low_dep_screener = original_df[original_df['num_dep_screener_0'] >= 9].copy()
    data_high_dep_screener = original_df[original_df['num_dep_screener_0'] < 9].copy()
    
    x_train_low, x_test_low, y_train_low, y_test_low = get_model_data(
        original_df = data_low_dep_screener,
        columns = columns_to_use_low
    )
    
    x_train_high, x_test_high, y_train_high, y_test_high = get_model_data(
        original_df = data_high_dep_screener,
        columns = columns_to_use_high
    )
    
    return x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high

x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_low,
    columns_to_use_high = model_features_opt2
)

print(x_train_low.shape)
print(x_test_low.shape)
print(y_train_low.shape)
print(y_test_low.shape)
print(x_train_high.shape)
print(x_test_high.shape)
print(y_train_high.shape)
print(y_test_high.shape)
print(len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high))

## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

gnb = GaussianNB()
gnb.fit(x_train_high, y_train_high)
y_pred_high = gnb.predict(x_test_high)

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)

gnb_low = GaussianNB()
gnb_low.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = gnb_low.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt6 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt6[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(2677, 9)
(670, 9)
(2677,)
(670,)
(3515, 29)
(879, 29)
(3515,)
(879,)
7741


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.957539,0.801305,0.872483,0.791478,1379.0
1,All Obs,Depressed (Yes),0.306329,0.711765,0.428319,0.791478,170.0
2,All Obs,Macro Avg,0.631934,0.756535,0.650401,0.791478,1549.0
3,All Obs,Weighted Avg,0.886070,0.791478,0.823737,0.791478,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.973199,0.907813,0.939369,0.888060,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.191781,0.466667,0.271845,0.888060,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.582490,0.687240,0.605607,0.888060,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.938210,0.888060,0.909480,0.888060,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.940754,0.709066,0.808642,0.717861,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.332298,0.764286,0.463203,0.717861,140.0


# Baseline For Comparison

In [18]:
## #####################################################
## Build Model

x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols,
    return_indices = True
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

bnb_base = BernoulliNB()
bnb_base.fit(x_train, y_train)
bnb_base_pred  = bnb_base.predict(x_test)

## #####################################################
## Get Features to identify performance across low & high dep screener response

bnb_base_pred_prob = pd.DataFrame(bnb_base.predict_proba(x_test))
bnb_base_pred_prob.columns = ['depressed_no_prob','depressed_yes_prob']
bnb_base_pred_prob['y_pred'] = bnb_base_pred
bnb_base_pred_prob['y_actual'] = y_test

bnb_base_pred_prob_with_feat = pd.concat([
    bnb_base_pred_prob, 
    # column bind test observations with their original values
    cdc_survey_pmom.iloc[idx_test][dep_screener_cols].reset_index()
], axis=1)

bnb_base_pred_prob_with_feat['num_dep_screener_0'] = (bnb_base_pred_prob_with_feat[dep_screener_cols]==0).sum(axis=1)

# 0 dep screener response
bnb_base_pred_low = bnb_base_pred_prob_with_feat[
    bnb_base_pred_prob_with_feat['num_dep_screener_0'] >= 9
]

# 1+ dep screener response
bnb_base_pred_high = bnb_base_pred_prob_with_feat[
    bnb_base_pred_prob_with_feat['num_dep_screener_0'] < 9
]

## #####################################################
## Cal scores

bnb_base_score_all = get_performance_df(y_test, bnb_base_pred,'All Obs')  

bnb_base_score_high = get_performance_df(
    label_actual = bnb_base_pred_high['y_actual'], 
    label_pred = bnb_base_pred_high['y_pred'],
    model_name = 'Obs w/ 1+ Dep Screener'
)

bnb_base_score_low = get_performance_df(
    label_actual = bnb_base_pred_low['y_actual'], 
    label_pred = bnb_base_pred_low['y_pred'],
    model_name = 'Obs w/ 0 Dep Screener'
)

baseline_performance = pd.concat([
    bnb_base_score_all,
    bnb_base_score_high,
    bnb_base_score_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
baseline_performance[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(6192, 10)
(1549, 10)
(6192,)
(1549,)


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.936236,0.866187,0.899851,0.826985,1390.0
1,All Obs,Depressed (Yes),0.292776,0.484277,0.364929,0.826985,159.0
2,All Obs,Macro Avg,0.614506,0.675232,0.632390,0.826985,1549.0
3,All Obs,Weighted Avg,0.870187,0.826985,0.844942,0.826985,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.956076,1.000000,0.977545,0.956076,653.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.000000,0.000000,0.000000,0.956076,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.478038,0.500000,0.488772,0.956076,683.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.914082,0.956076,0.934607,0.956076,683.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.913765,0.747626,0.822388,0.725173,737.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.292776,0.596899,0.392857,0.725173,129.0


# Data Dump For Comparison

In [19]:
best_performance_melt = ensemble_performance_opt5.melt(id_vars=['model','grouping'])
best_performance_melt['context'] = 'best'
baseline_performance_melt = baseline_performance.melt(id_vars=['model','grouping'])
baseline_performance_melt['context'] = 'baseline'
performance_all = pd.concat([best_performance_melt,baseline_performance_melt])
performance_all.to_csv("best_vs_baseline_performance.csv")